In [1]:
# pip3 install google-cloud-bigquery google-cloud-bigquery-storage pyarrow tqdm ipywidgets pandas matplotlib db-dtypes pandas-gbq

In [2]:
from google.cloud import bigquery

In [3]:
bq = bigquery.Client()

In [ ]:
# project => dataset => table

In [4]:
# what datasets do I have in my project?

In [6]:
for ds in bq.list_datasets("cs544-f23"):
    print(ds.dataset_id)

test1
test2
test3


In [9]:
ds = bigquery.Dataset("cs544-f23.test4")
# ds.location = "us-central1"
ds

Dataset(DatasetReference('cs544-f23', 'test4'))

In [13]:
bq.create_dataset(ds, exists_ok=True)

Dataset(DatasetReference('cs544-f23', 'test4'))

In [15]:
for ds in bq.list_datasets("bigquery-public-data"):
    print(ds.dataset_id)

america_health_rankings
austin_311
austin_bikeshare
austin_crime
austin_incidents
austin_waste
baseball
bbc_news
bigqueryml_ncaa
bitcoin_blockchain
blackhole_database
blockchain_analytics_ethereum_mainnet_us
bls
bls_qcew
breathe
broadstreet_adi
catalonian_mobile_coverage
catalonian_mobile_coverage_eu
census_bureau_acs
census_bureau_construction
census_bureau_international
census_bureau_usa
census_opportunity_atlas
census_utility
cfpb_complaints
chicago_crime
chicago_taxi_trips
clemson_dice
cloud_storage_geo_index
cms_codes
cms_medicare
cms_synthetic_patient_data_omop
country_codes
covid19_aha
covid19_covidtracking
covid19_ecdc
covid19_ecdc_eu
covid19_genome_sequence
covid19_geotab_mobility_impact
covid19_geotab_mobility_impact_eu
covid19_google_mobility
covid19_google_mobility_eu
covid19_govt_response
covid19_italy
covid19_italy_eu
covid19_jhu_csse
covid19_jhu_csse_eu
covid19_nyt
covid19_open_data
covid19_open_data_eu
covid19_public_forecasts
covid19_public_forecasts_asia_ne1
covid19_r

In [16]:
for t in bq.list_tables("bigquery-public-data.github_repos"):
    print(t.table_id)

commits
contents
files
languages
licenses
sample_commits
sample_contents
sample_files
sample_repos


In [17]:
%load_ext google.cloud.bigquery

In [19]:
%%bigquery
SELECT *
FROM `bigquery-public-data.github_repos.languages`
LIMIT 5

Query is running:   0%|          |

Downloading:   0%|          |

,repo_name,language
0,lemi136/puntovent,"[{'name': 'C', 'bytes': 80}]"
1,taxigps/nctool,"[{'name': 'C', 'bytes': 4461}]"
2,ahy1/strbuf,"[{'name': 'C', 'bytes': 5573}]"
3,nleiten/mod_rpaf-ng,"[{'name': 'C', 'bytes': 30330}]"
4,kmcallister/alameda,"[{'name': 'C', 'bytes': 17077}]"


In [20]:
%%bigquery df
SELECT *
FROM `bigquery-public-data.github_repos.languages`
LIMIT 5

Query is running:   0%|          |

Downloading:   0%|          |

In [21]:
df

,repo_name,language
0,lemi136/puntovent,"[{'name': 'C', 'bytes': 80}]"
1,taxigps/nctool,"[{'name': 'C', 'bytes': 4461}]"
2,ahy1/strbuf,"[{'name': 'C', 'bytes': 5573}]"
3,nleiten/mod_rpaf-ng,"[{'name': 'C', 'bytes': 30330}]"
4,kmcallister/alameda,"[{'name': 'C', 'bytes': 17077}]"


In [32]:
no_cache = bigquery.QueryJobConfig(use_query_cache=False)

In [33]:
q = bq.query("""
SELECT *
FROM `bigquery-public-data.github_repos.languages`
LIMIT 5
""", job_config=no_cache)

In [34]:
q.to_dataframe()

,repo_name,language
0,lemi136/puntovent,"[{'name': 'C', 'bytes': 80}]"
1,taxigps/nctool,"[{'name': 'C', 'bytes': 4461}]"
2,ahy1/strbuf,"[{'name': 'C', 'bytes': 5573}]"
3,nleiten/mod_rpaf-ng,"[{'name': 'C', 'bytes': 30330}]"
4,kmcallister/alameda,"[{'name': 'C', 'bytes': 17077}]"


In [35]:
q.total_bytes_processed / 1024**2 # MB

194.93627738952637

In [36]:
q.total_bytes_billed / 1024**2 # MB

195.0

In [27]:
# how many times can we do this in the free tier?

In [37]:
tb = 1024**4
tb / q.total_bytes_billed

5377.3128205128205

In [ ]:
# how much will it per query after that in, Tokyo

In [38]:
price_per_tb = 7.5
q.total_bytes_billed / tb * price_per_tb

0.0013947486877441406

In [ ]:
# pricing factors:
# 1. you pay for storage too (not just queries)
# 2. they have a minimum of 10 MB per query
# 3. they round up to the nearest 1 MB per query

In [39]:
%%bigquery df
SELECT *
FROM `bigquery-public-data.github_repos.languages`
WHERE repo_name LIKE 'open-lambda/%'
LIMIT 5

Query is running:   0%|          |

Downloading:   0%|          |

In [40]:
df

,repo_name,language
0,open-lambda/open-lambda,"[{'name': 'C', 'bytes': 31159}, {'name': 'Dock..."
1,open-lambda/load-balancer,"[{'name': 'C', 'bytes': 210}, {'name': 'Go', '..."
